<center> <h1>AI Agents : Powering LLMs With Web Search</h1>

### 1. Web Search Tool

#### 1.1 Tavily Search

In [1]:
%pip install langchain-tavily

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import yaml

with open('secrets.yml', 'r') as file:
    credentials = yaml.safe_load(file)

In [4]:
from langchain_tavily import TavilySearch

search_tool = TavilySearch(
    max_results=5,
    topic="general",
    tavily_api_key=credentials["tavily"]["api_key"],
    time_range="week",
)

#### 1.2 Usage

In [5]:
search_tool.invoke("Who are the leading scorers in the NBA for November 2025?")

{'query': 'Who are the leading scorers in the NBA for November 2025?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.espn.com/nba/dailyleaders/_/sort/threePointFieldGoalsAttempted/gameId/401810101',
   'title': 'NBA Daily Stats Leaders - November 17, 2025 - ESPN',
   'content': 'Game Leaders - November 17, 2025 ; N. Reid · 22 · 2 ; R. Gobert · 15 · 4 ; J. McDaniels · 15 · 3 ; D. DiVincenzo · 14 · 2 ; B. Williams · 15 · 2.',
   'score': 0.9999001,
   'raw_content': None},
  {'url': 'https://www.espn.com/nba/dailyleaders/_/sort/points/gameId/401810105',
   'title': 'NBA Daily Stats Leaders - November 18, 2025 - ESPN',
   'content': 'Game Leaders - November 18, 2025 ; 1, J. Brown, Bos ; 2, M. Porter Jr. BKN ; 3, P. Pritchard, Bos ; 4, D. Sharpe, BKN ; 5, D. White, Bos',
   'score': 0.9998591,
   'raw_content': None},
  {'url': 'https://sportsdata.usatoday.com/basketball/nba/players/stats/leaders',
   'title': '2025-26 NBA Player Stats Leag

### 2. Web-Powered Agent

#### 2.1 Langgraph Agent

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [ ]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="us.anthropic.claude-3-5-sonnet-20240620-v1:0",
    region_name="us-east-1",
    aws_access_key_id=credentials["bedrock"]["access_key"],
    aws_secret_access_key=credentials["bedrock"]["secret_key"]
)

class LLMNode:
    def __init__(self, llm):
        self.llm = llm

    def __call__(self, state: State):
        return {"messages": [self.llm.invoke(state["messages"])]}

llm_node = LLMNode(llm.bind_tools([search_tool]))

In [ ]:
from langgraph.prebuilt import ToolNode

tool_node = ToolNode([search_tool])

In [ ]:
from langgraph.graph import START
from langgraph.prebuilt import tools_condition

graph_builder.add_node("llm", llm_node)
graph_builder.add_node("tools", tool_node)


graph_builder.add_edge(START, "llm")
graph_builder.add_conditional_edges("llm", tools_condition) 
graph_builder.add_edge("tools", "llm")

In [ ]:
agent = graph_builder.compile()

In [ ]:
from IPython.display import Image, display
display(Image(agent.get_graph().draw_mermaid_png()))

#### 2.2 Usage

In [ ]:
for m in llm.stream("What is Anthropic's MCP?"):
    print(m.content, end="")

In [ ]:
for step in agent.stream(
    {"messages": "What is Anthropic's MCP?"},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()